In [ ]:
#working fashion mnist
# Fashion MNIST Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision as tv
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import os

train_batch_size = 64
test_batch_size = 1000

class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.conv1 = nn.Conv2d(1, 6, 5, stride=1, padding=2)
       self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
       self.fc1 = nn.Linear(16*5*5, 120)
       self.fc2 = nn.Linear(120, 84)
       self.fc3 = nn.Linear(84, 10)  # Change from 10 to 10 for Fashion-MNIST
   def forward(self, x):
       x = F.max_pool2d(F.relu(self.conv1(x)), 2)
       x = F.max_pool2d(F.relu(self.conv2(x)), 2)
       x = x.view(-1, self.num_flat_features(x))
       x = F.relu(self.fc1(x))
       x = F.relu(self.fc2(x))
       x = self.fc3(x)
       return x
   def num_flat_features(self, x):
       x_size = x.size()[1:]
       num = 1
       for n in x_size:
           num *= n
       return num

def load_data():
   transform = transforms.Compose([
       transforms.ToTensor(),
       transforms.Normalize((0.5,), (0.5,))  # Adjust normalization for Fashion-MNIST
   ])
   train_set = tv.datasets.FashionMNIST(
       root='./data',
       train=True,
       download=True,
       transform=transform
   )
   train_loader = torch.utils.data.DataLoader(
       train_set,
       batch_size=train_batch_size,
       shuffle=True,
       num_workers=2
   )
   test_set = tv.datasets.FashionMNIST(
       root='./data',
       train=False,
       download=True,
       transform=transform
   )
   test_loader = torch.utils.data.DataLoader(
       test_set,
       batch_size=test_batch_size,
       shuffle=False,
       num_workers=2
   )
   print("Data loaded successfully...")
   return train_loader, test_loader

def accuracy(model, x):
   with torch.no_grad():
       correct = 0
       total = 0
       for data in x:
           images, labels = data
           outputs = model(images)
           _, predicted = torch.max(outputs.data, 1)
           total += labels.size(0)
           correct += (predicted == labels).sum().item()

       return (100 * correct / total)

def train(train_loader, test_loader, model, criterion, optimizer, epoch):
   model.train()
   running_loss = 0
   for i, data in enumerate(train_loader, 0):
       inputs, labels = data
       optimizer.zero_grad()
       outputs = model(inputs)
       loss = criterion(outputs, labels)
       loss.backward()
       optimizer.step()

       running_loss += loss.item()
       if i % 200 == 199:
           print("[epoch %d, iter %5d] loss: %.3f" % (epoch+1, i+1, running_loss / 200))
           running_loss = 0.0
   train_acc = accuracy(model, train_loader)
   test_acc = accuracy(model, test_loader)
   print("epoch %d: train_acc %.3f, test_acc %.3f" % (epoch+1, train_acc, test_acc))
   return train_acc, test_acc

def display(train_acc, test_acc):
   fig, ax = plt.subplots()
   ax.plot(range(1, len(train_acc) + 1), train_acc, color='r', label='train_acc')
   ax.plot(range(1, len(test_acc) + 1), test_acc, color='b', label='test_acc')
   ax.legend(loc='lower right')
   plt.show()

if __name__ == '__main__':
   # input Fashion-MNIST
   train_loader, test_loader = load_data()
   # new model
   net = Net()

   # training
   learning_rate = 0.001
   momentum = 0.9
   max_epoch = 20
   criterion = nn.CrossEntropyLoss()
   optimizer = optim.Adam(net.parameters(), lr=learning_rate)

   train_acc = []
   test_acc = []
   for epoch in range(max_epoch):
       train_acc_t, test_acc_t = train(train_loader, test_loader, net, criterion, optimizer, epoch)
       train_acc.append(train_acc_t)
       test_acc.append(test_acc_t)

   display(train_acc, test_acc)

In [ ]:
# EE569 Homework Assignment #5 Sample Code
# requirements: python3 + pytorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision as tv
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import os

train_batch_size=64
test_batch_size=1000

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,6,5,stride=1,padding=2)
        self.conv2=nn.Conv2d(6,16,5,stride=1,padding=0)
        self.fc1=nn.Linear(16*5*5,120)
        self.fc2=nn.Linear(120,84)
        self.fc3=nn.Linear(84,10)
    def forward(self,x):
        x=F.max_pool2d(F.relu(self.conv1(x)),2)
        x=F.max_pool2d(F.relu(self.conv2(x)),2)
        x=x.view(-1,self.num_flat_features(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        return x
    def num_flat_features(self,x):
        #x is a 4D tensor
        x_size=x.size()[1:]
        num=1
        for n in x_size:
            num*=n
        return num

def load_data():
    transform=transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
    train_set=tv.datasets.MNIST(
        root='./data',
        train=True,
        download=True,
        transform=transform
        )
    train_loader=torch.utils.data.DataLoader(
        train_set,
        batch_size=train_batch_size,
        shuffle=True,
        num_workers=2)
    test_set=tv.datasets.MNIST(
        root='./data',
        train=False,
        download=True,
        transform=transform
        )
    test_loader=torch.utils.data.DataLoader(
        test_set,
        batch_size=test_batch_size,
        shuffle=False,
        num_workers=2)
    print("data loaded successfully...")
    return train_loader,test_loader

def accuracy(model,x,neg=False):
    with torch.no_grad():
        correct=0
        total=0
        class_correct = list(0. for i in range(10))
        class_total = list(0. for i in range(10))
        for data in x:
            images,labels=data
            if neg:
              images=-images
            images,labels=images, labels
            outputs=model(images)
            _,predicted=torch.max(outputs.data,1)
            total+=labels.size(0)
            correct+=(predicted==labels).sum().item()

        return (100 * correct / total)

def train(train_loader, test_loader, model, criterion, optimizer,epoch):
    model.train()
    running_loss=0
    for i,data in enumerate(train_loader,0):
        inputs,labels=data
        inputs,labels=inputs, labels
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss+=loss.item()
        if i%200==199:
            print("[epoch %d, iter %5d] loss: %.3f"%(epoch+1,i+1,running_loss/200))
            running_loss=0.0
    train_acc=accuracy(model,train_loader)
    test_acc=accuracy(model,test_loader)
    print("epoch %d: train_acc %.3f, test_acc %.3f"%(epoch+1,train_acc,test_acc))
    return train_acc,test_acc

def display(train_acc,test_acc):
    fig,ax=plt.subplots()
    ax.plot(range(1,len(train_acc)+1),train_acc,color='r',label='train_acc')
    ax.plot(range(1,len(test_acc)+1),test_acc,color='b',label='test_acc')
    ax.legend(loc='lower right')
    plt.show()

if __name__ == '__main__':
    # input MNIST
    train_loader,test_loader=load_data()
    # new model
    net=Net()

    # training
    learning_rate=0.01
    momentum=0.9
    max_epoch=10
    criterion=nn.CrossEntropyLoss()
    optimizer=optim.SGD(net.parameters(),lr=learning_rate,momentum=momentum)

    train_acc=[]
    test_acc=[]
    for epoch in range(max_epoch):
        train_acc_t,test_acc_t=train(train_loader, test_loader, net, criterion, optimizer,epoch)
        train_acc.append(train_acc_t)
        test_acc.append(test_acc_t)

    display(train_acc,test_acc)

In [ ]:
# CIFAR-10 Dataset
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision as tv
import torchvision.transforms as transforms

import matplotlib.pyplot as plt
import numpy as np
import math
import os

from torch.optim.lr_scheduler import StepLR

train_batch_size = 64
test_batch_size = 1000

class Net(nn.Module):
   def __init__(self):
       super(Net, self).__init__()
       self.conv1 = nn.Conv2d(3, 6, 5)  # Adjust input channels to 3 for CIFAR-10
       self.conv2 = nn.Conv2d(6, 16, 5)
       self.fc1 = nn.Linear(16 * 5 * 5, 120)
       self.fc2 = nn.Linear(120, 84)
       self.fc3 = nn.Linear(84, 10)  # Adjust output to 10 for CIFAR-10

   def forward(self, x):
       x = F.max_pool2d(F.relu(self.conv1(x)), 2)
       x = F.max_pool2d(F.relu(self.conv2(x)), 2)
       x = x.view(-1, self.num_flat_features(x))
       x = F.relu(self.fc1(x))
       x = F.relu(self.fc2(x))
       x = self.fc3(x)
       return x

   def num_flat_features(self, x):
       size = x.size()[1:]
       num_features = 1
       for s in size:
           num_features *= s
       return num_features

def load_data():
   transform = transforms.Compose([
       transforms.Resize((32, 32)),  # Resize images to 32x32 for CIFAR-10
       transforms.ToTensor(),
       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Adjust normalization for CIFAR-10
   ])
   train_set = tv.datasets.CIFAR10(
       root='./data',
       train=True,
       download=True,
       transform=transform
   )
   train_loader = torch.utils.data.DataLoader(
       train_set,
       batch_size=train_batch_size,
       shuffle=True,
       num_workers=2
   )
   test_set = tv.datasets.CIFAR10(
       root='./data',
       train=False,
       download=True,
       transform=transform
   )
   test_loader = torch.utils.data.DataLoader(
       test_set,
       batch_size=test_batch_size,
       shuffle=False,
       num_workers=2
   )
   print("Data loaded successfully...")
   return train_loader, test_loader

def accuracy(model, data_loader):
   with torch.no_grad():
       correct = 0
       total = 0
       for data in data_loader:
           images, labels = data
           outputs = model(images)
           _, predicted = torch.max(outputs.data, 1)
           total += labels.size(0)
           correct += (predicted == labels).sum().item()

   return (100 * correct / total)

def train(train_loader, test_loader, model, criterion, optimizer, scheduler, max_epoch):
   train_acc = []
   test_acc = []
   for epoch in range(max_epoch):
       model.train()
       running_loss = 0
       for i, data in enumerate(train_loader, 0):
           inputs, labels = data
           optimizer.zero_grad()
           outputs = model(inputs)
           loss = criterion(outputs, labels)
           loss.backward()
           optimizer.step()

           running_loss += loss.item()
           if i % 200 == 199:
               print("[epoch %d, iter %5d] loss: %.3f" % (epoch+1, i+1, running_loss / 200))
               running_loss = 0.0


       # Update learning rate
       scheduler.step()

       train_acc_epoch = accuracy(model, train_loader)
       test_acc_epoch = accuracy(model, test_loader)
       train_acc.append(train_acc_epoch)
       test_acc.append(test_acc_epoch)
       print("epoch %d: train_acc %.3f, test_acc %.3f" % (epoch+1, train_acc_epoch, test_acc_epoch))

   return train_acc, test_acc

def display(train_acc, test_acc):
   fig, ax = plt.subplots()
   ax.plot(range(1, len(train_acc) + 1), train_acc, color='r', label='train_acc')
   ax.plot(range(1, len(test_acc) + 1), test_acc, color='b', label='test_acc')
   ax.legend(loc='lower right')
   plt.show()

if __name__ == '__main__':
   # Input CIFAR-10
   train_loader, test_loader = load_data()
   # New model
   net = Net()

   # Training
   learning_rate = 0.0008
   max_epoch = 30
   criterion = nn.CrossEntropyLoss()
   #optimizer = optimizer = optim.Adam(net.parameters(), lr=learning_rate)
   optimizer = optim.Adam(net.parameters(), lr=learning_rate, betas=(0.9, 0.999))  # You can adjust the betas parameter for momentum

 #Learning rate scheduler
   step_size = 15  # Step size for learning rate decay
   gamma = 0.1     # Multiplicative factor for learning rate decay
   scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

   train_acc, test_acc = train(train_loader, test_loader, net, criterion, optimizer, scheduler, max_epoch)

   display(train_acc, test_acc)